In [0]:
!pip install pymysql
import pymysql

host='49.247.130.147'
username='root'
password='jihyun0127'
db_name='kpc_test'
db= pymysql.connect(host=host, user=username, passwd=password, db=db_name, connect_timeout=1)

In [0]:
from sqlalchemy import create_engine

engine = create_engine("mysql+pymysql://"+username+":"+password+"@"+host+":3306/"+db_name+"?charset=utf8", encoding='utf-8')
conn = engine.connect()

In [0]:
import pandas as pd

movie1 = pd.read_sql_table('att_table', conn)
movie2 = pd.read_sql_table('keyword_table', conn)

movie1.head()

,id,name,star,num,type,address,lat,lon,location_code,image
0,320359,창덕궁,45,4610,T020303,"종로구 율곡로 99,",37.580095,126.991850,L0101,https://media-cdn.tripadvisor.com/media/photo-...
1,324887,덕수궁,40,1074,T020303,"중구 세종대로 99,",37.565871,126.975008,L0101,https://media-cdn.tripadvisor.com/media/photo-...
2,324888,경복궁,45,10649,T020101,"종로구 사직로 161,",37.577362,126.976684,L0101,https://media-cdn.tripadvisor.com/media/photo-...
3,324891,롯데월드,40,3111,T030108,"송파구 올림픽로 240,",37.511235,127.098027,L0101,https://media-cdn.tripadvisor.com/media/photo-...
4,324907,남대문시장,40,2894,T020204,"중구 남대문시장4길 21,",37.559147,126.977641,L0101,https://media-cdn.tripadvisor.com/media/photo-...


In [0]:
movies_df = pd.merge(movie1,movie2,on='id')

In [0]:
movies.head()

,id,name,star,num,type,address,lat,lon,location_code,image,valuation,view,cost,total
0,320359,창덕궁,45,4610,T020303,"종로구 율곡로 99,",37.580095,126.991850,L0101,https://media-cdn.tripadvisor.com/media/photo-...,traditional,historical,free,traditional/free/beautiful/amazing/great/histo...
1,324887,덕수궁,40,1074,T020303,"중구 세종대로 99,",37.565871,126.975008,L0101,https://media-cdn.tripadvisor.com/media/photo-...,interesting,big,free,free/interesting/modern/traditional/nice/beaut...
2,324888,경복궁,45,10649,T020101,"종로구 사직로 161,",37.577362,126.976684,L0101,https://media-cdn.tripadvisor.com/media/photo-...,traditional,historical,free,traditional/free/beautiful/historical/great/in...
3,324891,롯데월드,40,3111,T030108,"송파구 올림픽로 240,",37.511235,127.098027,L0101,https://media-cdn.tripadvisor.com/media/photo-...,great,extra,worth,great/extra/young/fun/worth/big/huge/nice/mode...
4,324907,남대문시장,40,2894,T020204,"중구 남대문시장4길 21,",37.559147,126.977641,L0101,https://media-cdn.tripadvisor.com/media/photo-...,interesting,big,cheap,interesting/cheap/traditional/nice/great/expen...


In [0]:
movies_df=movies_df[movies_df['star'] != 50]

In [0]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 121 entries, 0 to 127
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             121 non-null    int64  
 1   name           121 non-null    object 
 2   star           121 non-null    int64  
 3   num            121 non-null    int64  
 4   type           121 non-null    object 
 5   address        121 non-null    object 
 6   lat            121 non-null    float64
 7   lon            121 non-null    float64
 8   location_code  121 non-null    object 
 9   image          121 non-null    object 
 10  valuation      121 non-null    object 
 11  view           121 non-null    object 
 12  cost           121 non-null    object 
 13  total          121 non-null    object 
dtypes: float64(2), int64(3), object(9)
memory usage: 14.2+ KB


In [0]:
## 타입 count기반 피처 벡터화

from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer를 적용하기 위해 공백문자로 word 단위가 구분되는 문자열로 변환. 
#movies_df['ttype_literal'] = movies_df['ttype'].apply(lambda x : (' ').join(x))
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
genre_mat = count_vect.fit_transform(movies['total'])
print(genre_mat.shape)

(121, 720)


In [0]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[:2])

(121, 121)
[[1.         0.31578947 0.52631579 0.10526316 0.21052632 0.36842105
  0.31578947 0.21052632 0.21052632 0.10526316 0.26315789 0.26315789
  0.26315789 0.26315789 0.31578947 0.31578947 0.31578947 0.42105263
  0.26315789 0.36842105 0.36842105 0.26315789 0.21052632 0.36842105
  0.21052632 0.31578947 0.21052632 0.21052632 0.31578947 0.21052632
  0.26315789 0.15789474 0.26315789 0.26315789 0.31578947 0.10526316
  0.42105263 0.26315789 0.15789474 0.         0.26315789 0.31578947
  0.36842105 0.31578947 0.26315789 0.26315789 0.21052632 0.15789474
  0.21052632 0.31578947 0.15789474 0.26315789 0.10526316 0.26315789
  0.15789474 0.26315789 0.21052632 0.26315789 0.15789474 0.42105263
  0.21052632 0.21052632 0.31578947 0.26315789 0.12725695 0.21052632
  0.15789474 0.15789474 0.15789474 0.         0.15789474 0.15789474
  0.21052632 0.43355498 0.21052632 0.         0.42105263 0.21052632
  0.31578947 0.15789474 0.26315789 0.15789474 0.21052632 0.31578947
  0.26315789 0.26315789 0.21052632 0.

In [0]:
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1]
print(genre_sim_sorted_ind[:1])

[[  0   2  73  36  17  59  76   5  20  23  42  96  19  83  49  98  34  28
   25  16  15  14  78  41   1  62 104  43   6 112  55  51  53  45  30  33
   32  63  44  57 119  37  40  85 101  18 111  10  11  12  13  99  95  90
   21 115  84 117  80 108 107   7   4  46  26   8  27  22  29  24  48  60
   92  65  77  82  86  87  74  97  72  61  56 110  89 100  91 116  47  79
  118  50  81  58  31  70  68  71  54  67  66  38  94 102  64 105 113   3
   35  52 103   9 109  93 114  75 106  39  88  69 120]]


In [0]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    
    # 인자로 입력된 movies_df DataFrame에서 'title' 컬럼이 입력된 title_name 값인 DataFrame추출
    title_movie = df[df['name'] == title_name]
    
    # title_named을 가진 DataFrame의 index 객체를 ndarray로 반환하고 
    # sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n 개의 index 추출
    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]
    
    # 추출된 top_n index들 출력. top_n index는 2차원 데이터 임. 
    #dataframe에서 index로 사용하기 위해서 1차원 array로 변경
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    
    return df.iloc[similar_indexes]


In [0]:
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, '경복궁',10)
similar_movies[['name', 'star']]

[[ 2  0 59 17 49 42 57 76 53 36]]


,name,star
2,경복궁,45
0,창덕궁,45
59,경희궁,40
17,광화문,40
49,문화역서울 284,40
42,청와대,40
57,서울 역사 박물관,45
76,숭례문,40
53,신세계 백화점 본점,40
36,창경궁,45


In [0]:
movies_df[['name','star','num']].sort_values('star', ascending=False)[:10]

,name,star,num
0,창덕궁,45,4610
32,한강 공원,45,1569
105,서울 한양 도성,45,168
36,창경궁,45,736
103,응봉산,45,21
101,관악산,45,31
40,예술의전당,45,882
43,서울글로벌문화체험센터,45,52
98,두물머리,45,112
45,이화여자대학교,45,913


In [0]:
C = movies_df['star'].mean()
m = movies_df['num'].quantile(0.6)
print('C:',round(C,3), 'm:',round(m,3))

C: 42.066 m: 368.0


In [0]:
percentile = 0.6
m = movies_df['num'].quantile(percentile)
C = movies_df['star'].mean()

def weighted_vote_average(record):
    v = record['num']
    R = record['star']
    
    return ( (v/(v+m)) * R ) + ( (m/(m+v)) * C )   

movies_df['weighted_star'] = movies_df.apply(weighted_vote_average, axis=1) 

In [0]:
movies_df[['name','star','weighted_star','num']].sort_values('weighted_star',
                                                                          ascending=False)[:10]


,name,star,weighted_star,num
2,경복궁,45,44.902000,10649
9,명동 쇼핑 거리,45,44.899294,10353
0,창덕궁,45,44.783112,4610
12,한국 전쟁 기념관,45,44.767363,4273
27,청계천,45,44.749380,3940
5,국립중앙박물관,45,44.604517,2362
48,홍대앞 거리,45,44.588541,2256
14,남산 공원,45,44.581361,2211
123,쿠킹 난타,45,44.519934,1881
32,한강 공원,45,44.442607,1569


In [0]:
## 특정 관광지에 대한 컨텐츠 기반 추천

def find_sim_movie(title_name, top_n=10):
  try: 
    title_movie = movies_df[movies_df['name'] == title_name]
    title_index = title_movie.index.values

    # top_n의 2배에 해당하는 쟝르 유사성이 높은 index 추출 
    similar_indexes = genre_sim_sorted_ind[title_index, :(top_n*2)]
    similar_indexes = similar_indexes.reshape(-1)
    # 기준 영화 index는 제외
    answer = movies_df.iloc[similar_indexes].sort_values('weighted_star', ascending=False)[:top_n]
    answer = answer[answer['name']!=title_name]
    # top_n의 2배에 해당하는 후보군에서 weighted_star 높은 순으로 top_n 만큼 추출 
    return answer
  except:
    print('추천 관광지 없음')
  ### 오류 나면 loc -> iloc로 변경해보기

In [0]:
similar_movies = find_sim_movie('경복궁',10)

similar_movies[['name', 'star', 'weighted_star','num','total']]

,name,star,weighted_star,num,total
0,창덕궁,45,44.783112,4610,traditional/free/beautiful/amazing/great/histo...
5,국립중앙박물관,45,44.604517,2362,free/great/beautiful/interesting/special/nice/...
19,봉은사,45,44.351550,1297,beautiful/peaceful/great/big/free/amazing/nice...
111,롯데월드타워&몰,45,44.075625,800,great/nice/amazing/local/incredible/expensive/...
36,창경궁,45,44.022039,736,nice/beautiful/traditional/free/peaceful/inter...
57,서울 역사 박물관,45,43.409912,311,free/great/interesting/nice/modern/big/impress...
33,인왕산,45,42.590024,80,great/beautiful/hot/big/amazing/worth/interest...
59,경희궁,40,41.574183,115,free/beautiful/peaceful/nice/amazing/interesti...
16,서울 월드컵 경기장 (상암 월드컵 경기장),40,41.338610,200,great/nice/interesting/cheap/big/interactive/c...
